In [1]:
pip install langchain langchain-community langchain_openai langchain_text_splitters sentence_transformers pypdfium2

  Using cached sentence_transformers-4.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached torch-2.7.0-cp313-cp313-win_amd64.whl.metadata (29 kB)
  Using cached scikit_learn-1.6.1-cp313-cp313-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.15.2-cp313-cp313-win_amd64.whl.metadata (60 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached sentence_transformers-4.1.0-py3-none-any.whl (345 kB)
   ---------------------------------------- 0.0/3.0 MB ? eta -:--:--
   ----------------- ---------------------- 1.3/3.0 MB 7.5 MB/s eta 0:00:01
   ----------------------------------- ---- 2.6/3.0 MB 6.8 MB/s eta 0:00:01
   ---------------------------------------- 3.0/3.0 MB 6.5 MB/s eta 0:00:00
Using cached torch-2.7.0-cp313-cp313-win_amd64.whl (212.5 MB)
U


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
!pip install chromadb
!pip install langchain langchain-openai


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
!pip install -U langchain-chroma

  Using cached langchain_chroma-0.2.3-py3-none-any.whl.metadata (1.1 kB)
  Using cached chromadb-0.6.3-py3-none-any.whl.metadata (6.8 kB)
Using cached langchain_chroma-0.2.3-py3-none-any.whl (11 kB)
Using cached chromadb-0.6.3-py3-none-any.whl (611 kB)
  Attempting uninstall: chromadb
    Found existing installation: chromadb 1.0.4
    Uninstalling chromadb-1.0.4:
      Successfully uninstalled chromadb-1.0.4


  You can safely remove it manually.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import shutil
import os
from pathlib import Path

# 1. 프로젝트 폴더 내 Chroma 디렉토리 삭제
shutil.rmtree("./chroma_db", ignore_errors=True)

# 2. 사용자 홈 폴더 아래 Chroma 캐시 삭제
shutil.rmtree(Path.home() / ".chromadb", ignore_errors=True)

# 3. Windows LocalAppData 경로 삭제 (숨겨진 문제 흔함)
shutil.rmtree(Path.home() / "AppData" / "Local" / "chromadb", ignore_errors=True)

print("✅ 모든 Chroma DB 경로 완전 삭제 완료 — 커널 재시작 후 계속하세요")

✅ 모든 Chroma DB 경로 완전 삭제 완료 — 커널 재시작 후 계속하세요


In [2]:
import shutil
import os
from pathlib import Path

# 전체 Chroma DB 삭제 (기본 디렉토리와 시스템 캐시 포함)
shutil.rmtree("./chroma_db", ignore_errors=True)
shutil.rmtree(Path.home() / ".chromadb", ignore_errors=True)

In [3]:
import os
import json
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
import chromadb

os.environ["OPENAI_API_KEY"] = "키 넣으세용"

embeddings_model = OpenAIEmbeddings(model="text-embedding-3-small")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
client = chromadb.PersistentClient(path="./chroma_db")

def load_json_documents(json_files):
    docs = []
    for file in json_files:
        with open(file, "r", encoding="utf-8") as f:
            data = json.load(f)
        for item in data:
            if "Text" in item and "Completion" in item:
                docs.append(Document(
                    page_content=f"질문: {item['Text']}\n답변: {item['Completion']}",
                    metadata={"카테고리": item.get("카테고리", "")}
                ))
            elif "lecture_name" in item and "student_id" in item:
                content = (
                    f"학번: {item['student_id']}\n"
                    f"과목명: {item['lecture_name']}\n"
                    f"학정번호: {item['lecture_id']}\n"
                    f"개설 학과: {item['department_offered']}\n"
                    f"이수 구분: {item['lecture_course_type']}\n"
                    f"학점: {item['lecture_credit']}학점\n"
                    f"성적: {item['lecutre_grade']}"
                )
                if "retake_or_delete_status" in item:
                    content += f"\n재수강/삭제 여부: {item['retake_or_delete_status']}"
                if "retake_status" in item:
                    content += f"\n재수강 여부: {item['retake_status']}"

                docs.append(Document(page_content=content))
    return docs


In [5]:
user_datasets = {
    "kim": ["data/kw_chatbot_data - 김브티_수강이력.json"],
    "hong": ["data/kw_chatbot_data - 홍데사_수강이력.json"]
}

task_datasets = {
    "lecture_search": [
        "data/kw_chatbot_data - Student.json",
        "data/kw_chatbot_data - 강의 평점.json",
        "data/kw_chatbot_data - 강의계획서.json",
        "data/kw_chatbot_data - 김브티_성적.json",
        "data/kw_chatbot_data - 김브티_수강이력.json",
        "data/kw_chatbot_data - 수강신청자료집.json",
        "data/kw_chatbot_data - 커리큘럼(DS).json",
        "data/kw_chatbot_data - 커리큘럼(VT).json",
        "data/kw_chatbot_data - 홍데사_성적.json",
        "data/kw_chatbot_data - 홍데사_수강이력.json"
    ],
    "career_counsel": ["data/진로상담.json"],
    "academic_status": ["data/학습현황.json"]
}

# 사용자별 컬렉션
for user_id, files in user_datasets.items():
    collection_name = f"lecture_search_{user_id}"
    try:
        client.delete_collection(name=collection_name)
    except:
        pass
    docs = load_json_documents(files)
    split_docs = text_splitter.split_documents(docs)
    texts = [doc.page_content for doc in split_docs]
    embeddings = embeddings_model.embed_documents(texts)
    collection = client.create_collection(name=collection_name)
    collection.add(
        documents=texts,
        embeddings=embeddings,
        ids=[f"{collection_name}_{i}" for i in range(len(texts))]
    )
    print(f"[{user_id}] 임베딩 완료 ({len(embeddings)}건)")

# 기능별 컬렉션
for task, files in task_datasets.items():
    try:
        client.delete_collection(name=task)
    except:
        pass
    docs = load_json_documents(files)
    split_docs = text_splitter.split_documents(docs)
    texts = [doc.page_content for doc in split_docs]
    embeddings = embeddings_model.embed_documents(texts)
    collection = client.create_collection(name=task)
    collection.add(
        documents=texts,
        embeddings=embeddings,
        ids=[f"{task}_{i}" for i in range(len(texts))]
    )
    print(f"[{task}] 임베딩 완료 ({len(embeddings)}건)")

[kim] 임베딩 완료 (24건)
[hong] 임베딩 완료 (24건)
[lecture_search] 임베딩 완료 (48건)
[career_counsel] 임베딩 완료 (44건)
[academic_status] 임베딩 완료 (40건)


In [35]:
from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)

def get_collection_name(user_input: str, user_id: str) -> str:
    q = user_input.lower()
    if "진로" in q or "상담" in q:
        return "career_counsel"
    elif "이수" in q or "학점" in q or "현황" in q or "평균" in q:
        return "academic_status"
    return f"lecture_search_{user_id}"

def answer_query(user_input: str, user_id: str = "kim"):
    collection_name = get_collection_name(user_input, user_id)
    vectorstore = Chroma(
        persist_directory="./chroma_db",
        collection_name=collection_name,
        embedding_function=embeddings_model
    )
    prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""
        다음은 사용자의 수강 이력 또는 학업/진로 관련 정보입니다.
        질문에 답할 때 반드시 정확한 정보를 바탕으로 하세요.

        성적은 A+, A0, B+, B0, C+, C0, F 등의 형식이며, 
        ❗ 절대 유사한 등급을 포함하거나 추론하지 마세요.
        예: "A+"를 요청했을 경우, 반드시 성적이 "A+"인 과목만 포함하세요. "A0"는 포함하면 안 됩니다.

        📌 답변 형식 지침:
        - 질문에 해당하는 내용이 **여러 개인 경우 빠짐없이 모두 나열**하세요.
        - 질문 유형에 따라 말끝을 다르게 쓰세요.
          예:
            - "내가 A+ 받은 과목 알려줘" → "OOO, OOO 과목에서 A+를 받으셨습니다."
            - "자료구조 성적이 뭐야?" → "자료구조의 성적은 B+입니다."
            - "객체지향프로그래밍이 전선이지?" → "네, 객체지향프로그래밍은 전필입니다."
            - "전체 평균이 어떻게 돼?" → "전체 성적 평균은 3.58입니다."
        - **절대로 "답변:"으로 시작하지 마세요. 문장은 바로 시작하세요.**
        - 정중하고 간결하게, 핵심만 자연스럽게 응답하세요.
        - 축하 인사, 추가 질문 권유 등은 하지 마세요.

        문서:
        {context}

        질문:
        {question}

        위 지침을 따라 **답변 접두어 없이** 자연스럽게 말문을 여세요.
        """
    )
    qa = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectorstore.as_retriever(search_kwargs={"k": 20}),
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt}
    )
    return qa.invoke({"query": user_input})["result"]

In [34]:
##성적추가한 거...아직 정확하진 않은듯..?
## 근데 그 성적 따로 올릴거니까 상관없지않나..? 일단 해봄

from langchain_chroma import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)

def get_collection_name(user_input: str, user_id: str) -> str:
    q = user_input.lower()
    if "진로" in q or "상담" in q:
        return "career_counsel"
    elif "이수" in q or "학점" in q or "현황" in q or "평균" in q:
        return "academic_status"
    return f"lecture_search_{user_id}"

def answer_query(user_input: str, user_id: str = "kim"):
    if "평균" in user_input or "전선 성적" in user_input:
        file_path = f"data/kw_chatbot_data - {'김브티' if user_id == 'kim' else '홍데사'}_수강이력.json"
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        grades = {
            "A+": 4.5, "A0": 4.0,
            "B+": 3.5, "B0": 3.0,
            "C+": 2.5, "C0": 2.0,
            "D+": 1.5, "D0": 1.0,
            "F": 0.0
        }

        total_points = 0
        total_credits = 0
        for item in data:
            grade = item.get("lecutre_grade")
            credit = item.get("lecture_credit", 0)
            is_major = item.get("lecture_course_type") == "전선"
            if grade in grades and isinstance(credit, (int, float)):
                if "전선" in user_input:
                    if is_major:
                        total_points += grades[grade] * credit
                        total_credits += credit
                else:
                    total_points += grades[grade] * credit
                    total_credits += credit

        if total_credits == 0:
            return "성적 정보가 없습니다."

        gpa = round(total_points / total_credits, 2)
        if "전선" in user_input:
            return f"전선 성적 평균은 {gpa}입니다."
        return f"전체 성적 평균은 {gpa}입니다."

    collection_name = get_collection_name(user_input, user_id)
    vectorstore = Chroma(
        persist_directory="./chroma_db",
        collection_name=collection_name,
        embedding_function=embeddings_model
    )

    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template="""
        다음은 사용자의 수강 이력 또는 학업/진로 관련 정보입니다.
        질문에 답할 때 반드시 정확한 정보를 바탕으로 하세요.

        성적은 A+, A0, B+, B0, C+, C0, F 등의 형식이며,
        ❗ 절대 유사한 등급을 포함하거나 추론하지 마세요.
        예: "A+"를 요청했을 경우, 반드시 성적이 "A+"인 과목만 포함하세요. "A0"는 포함하면 안 됩니다.

        📌 답변 형식 지침:
        - 질문에 해당하는 내용이 **여러 개인 경우 빠짐없이 모두 나열**하세요.
        - 질문 유형에 따라 말끝을 다르게 쓰세요.
          예:
            - "내가 A+ 받은 과목 알려줘" → "OOO, OOO 과목에서 A+를 받으셨습니다."
            - "자료구조 성적이 뭐야?" → "자료구조의 성적은 B+입니다."
            - "객체지향프로그래밍이 전선이지?" → "네, 객체지향프로그래밍은 전필입니다."
            - "전체 평균이 어떻게 돼?" → "전체 성적 평균은 3.58입니다."
            - "F 받은 과목은?" → "OOO, OOO 과목에서 F를 받으셨습니다."
            - "B 이상 받은 과목은?" → "OOO, OOO 과목에서 B 이상을 받으셨습니다."
            - "C+ 받은 과목은?" → "OOO 과목에서 C+를 받으셨습니다."
            - "재수강하고 학점이 어떻게 바꼈어?" → "OOO 과목은 B0에서 A0로 바뀌었습니다."
        - **절대로 '답변:'으로 시작하지 마세요. 문장은 바로 시작하세요.**
        - 정중하고 간결하게, 핵심만 자연스럽게 응답하세요.
        - 축하 인사, 추가 질문 권유 등은 하지 마세요.

        문서:
        {context}

        질문:
        {question}

        위 지침을 따라 **답변 접두어 없이** 자연스럽게 말문을 여세요.
        """
    )

    qa = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=vectorstore.as_retriever(search_kwargs={"k": 20}),
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt}
    )

    return qa.invoke({"query": user_input})["result"]

In [36]:
answer_query("인터랙티브미디어개론 학점이 어때?", user_id="kim")

'인터랙티브미디어개론의 학점은 B+입니다.'

In [37]:
answer_query("내가 A+ 받은 과목 알려줘", user_id="kim")

'인터넷활용, 대학수학및연습2, 경제와경영, 생활속의생명과학 중에서 A+를 받은 과목입니다.'

In [38]:
answer_query("내가 A+ 받은 과목 알려줘", user_id="hong")

'AI수학, 그래픽디자인, 대학수학및연습2, 대학수학및연습1, 컴퓨팅사고, 생활속의생명과학, 화폐와금융의과거,현재그리고미래에서 A+를 받았습니다.'

In [14]:
answer_query("재수강한 거 뭐있어?", user_id="hong")

'창의설계입문 과목을 재수강했습니다.'

In [23]:
answer_query("재수강하고 학점이 어떻게 바꼈어?", user_id="hong")

'프로그래밍 기초 과목은 B0에서 A+로 바뀌었습니다.'

In [30]:
answer_query("재수강하고 전체학점이 어떻게 바꼈어?", user_id="hong")

'재수강한 과목인 프로그래밍 기초는 B0에서 A0로 바뀌어 전체 학점이 상승했습니다.'

In [24]:
answer_query("전체 성적 평균이 어떻게 돼?", user_id="hong")

'전체 성적 평균은 3.62입니다.'

In [26]:
answer_query("전체 성적 평균이 어떻게 돼?", user_id="kim")

'전체 성적 평균은 3.65입니다.'

In [31]:
answer_query("전공 평균이 어떻게 돼?", user_id="kim")

'전체 성적 평균은 3.65입니다.'

In [19]:
answer_query("재수강 몇 번 했어?", user_id="hong")

'창의설계입문 과목을 한 번 재수강했습니다.'

In [27]:
answer_query("객체지향프로그래밍 성적이 뭐야?", user_id="kim")

'객체지향프로그래밍의 성적은 B0입니다.'